## Dependencies

In [ ]:
!pip install -q pandas kagglehub numpy scipy matplotlib scikit-learn ipykernel jupyter pytest nltk sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel==5.5.6, but you have ipykernel 6.29.5 which is incompatible.


In [ ]:
import os
import re

import kagglehub

import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sentence_transformers import SentenceTransformer

import nltk
from nltk.corpus import stopwords
nltk.download('punkt_tab')
nltk.download('stopwords')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Load Dataset and Models

In [ ]:

path = kagglehub.dataset_download("arshkon/linkedin-job-postings")
df = pd.read_csv(os.path.join(path,"postings.csv"))

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
t5_model = SentenceTransformer('sentence-transformers/sentence-t5-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

rust_model.ot:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

Clean Data


In [ ]:
salary_df = df[df["max_salary"].notna() & df["min_salary"].notna() & df["description"].notna()]

In [ ]:
## Annualize Salaries

salary_scale = {
    "HOURLY": 2080,
    "WEEKLY": 52,
    "BIWEEKLY": 26,
    "MONTHLY":12
}
for period, scale in salary_scale.items():
    salary_df.loc[salary_df['pay_period'] == period, 'max_salary'] *= scale
    salary_df.loc[salary_df['pay_period'] == period, 'min_salary'] *= scale
    salary_df.loc[salary_df['pay_period'] == period, 'pay_period'] = "YEARLY"

In [ ]:
## Create total_text column from concatenation over other text columns

salary_df["total_text"] = salary_df["title"] + ' ' + salary_df["description"] + ' ' + salary_df["location"]

<ipython-input-7-52018840db5a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salary_df["total_text"] = salary_df["title"] + ' ' + salary_df["description"] + ' ' + salary_df["location"]


T5 Embedding for all text data

In [ ]:
salary_df["total_text_embedding"] = salary_df["total_text"].apply(lambda x: t5_model.encode(x))

<ipython-input-8-c6d6b2f45fe7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salary_df["total_text_embedding"] = salary_df["total_text"].apply(lambda x: t5_model.encode(x))


In [ ]:
pd.DataFrame(salary_df.total_text_embedding.tolist(), index= salary_df.index).to_csv("linkedin/t5-total-result.csv", index=False)

T5 Embedding for Title

In [ ]:
salary_df["title_embedding"] = salary_df["title"].apply(lambda x: t5_model.encode(x))

In [ ]:
pd.DataFrame(salary_df.title_embedding.tolist(), index= salary_df.index).to_csv("linkedin_embeddings/t5-title-result.csv")

Paraphrase Embedding for Total Text

In [ ]:
salary_df["total_text_embedding"] = salary_df["total_text"].apply(lambda x: model.encode(x))

In [ ]:
pd.DataFrame(salary_df.total_text_embedding.tolist(), index= salary_df.index).to_csv("linkedin/paraphrase-total-result.csv", index=False)

Paraphrase Embedding for Title

In [ ]:
salary_df["title_embedding"] = salary_df["title"].apply(lambda x: model.encode(x))

In [ ]:
pd.DataFrame(salary_df.title_embedding.tolist(), index= salary_df.index).to_csv("linkedin/paraphrase-title-result.csv", index=False)

Count Vectorize Unigram

In [ ]:
unigram_vectorizer = CountVectorizer(
    max_features=1000,
    stop_words=stopwords.words('english'),
    token_pattern=r"[a-zA-Z]+"
)  # Limit to top 1000 words
word_counts = unigram_vectorizer.fit_transform(
    salary_df["description"],
)
unigram_df = pd.DataFrame(
    word_counts.toarray(),
    columns=unigram_vectorizer.get_feature_names_out()
)
unigram_df.head()

,abilities,ability,able,access,accommodation,accommodations,accordance,according,account,accounting,...,workplace,works,world,would,writing,written,www,year,years,york
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1,0,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
unigram_df.to_csv("linkedin/unigram.csv", index=False)

Count Vectorize Bigram

In [ ]:
bigram_vectorizer = CountVectorizer(
    analyzer='word',
    ngram_range=(1, 2),
    max_features=1000,
    stop_words="english",
    token_pattern=r"[a-zA-Z]+"
)
word_counts = bigram_vectorizer.fit_transform(
    salary_df["description"],
)
bigram_df = pd.DataFrame(
    word_counts.toarray(),
    columns=bigram_vectorizer.get_feature_names_out()
)

In [ ]:
bigram_df.to_csv("linkedin/bigram.csv", index=False)

Count Vectorize Unigram Laplace Smoothing

In [ ]:
smoothed_unigram = unigram_df + 1

In [ ]:
smoothed_unigram.to_csv("linkedin/smoothed_unigram.csv")

In [ ]:
smoothed_unigram.shape

(29792, 1000)

TFIDF

In [ ]:

tfidf_vectorizer = TfidfVectorizer(
    max_features=1000,
    stop_words=stopwords.words('english'),
    token_pattern=r"[a-zA-Z]+"
)
word_counts = tfidf_vectorizer.fit_transform(
    salary_df["total_text"]
)

tfidf_df = pd.DataFrame(
    word_counts.toarray(),
    columns=tfidf_vectorizer.get_feature_names_out()
)

In [ ]:
tfidf_df.to_csv("linkedin/tfidf.csv",index=False)